<div class="pull-right">
<a href="http://numfys.net/examples"><img src="https://www.numfys.net/static/img/favicon.ico" style="height:60px; margin-top: 25px;"></a>
</div>

# Простая фильтрация звука с использованием дискретного преобразования Фурье

### Example – Waves and Acoustics 
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes og Jon Andreas Støvneng.

Last edited: May 20th 2016

___

Эта тетрадь дает представление о работе со звуками на Python. Мы используем класс Python [wave](https://docs.python.org/2/library/wave.html). Изучите материал по ссылке, чтобы понять, как работают различные функции! Ближе к концу несколько звуковых дорожек фильтруются с использованием дискретных преобразований Фурье (DFT) и тригонометрического приближения наименьших квадратов.

**ПРИМЕЧАНИЕ: Некоторые звуковые файлы могут быть немного громкими, и перед их воспроизведением рекомендуется уменьшить громкость.**

Как всегда, мы начинаем с импорта необходимых библиотек и задаем общие параметры рисунка.

In [ ]:
import numpy as np
from matplotlib import pylab as plt
import wave
import IPython
from scipy import fftpack as fft
%matplotlib inline

# Casting unitary numbers to real numbers will give errors
# because of numerical rounding errors. We therefore disable 
# warning messages.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set common figure parameters
newparams = {'axes.labelsize': 8, 'axes.linewidth': 1, 'savefig.dpi': 200,
             'lines.linewidth': 1, 'figure.figsize': (8, 3),
             'ytick.labelsize': 7, 'xtick.labelsize': 7,
             'ytick.major.pad': 5, 'xtick.major.pad': 5,
             'legend.fontsize': 7, 'legend.frameon': True, 
             'legend.handlelength': 1.5, 'axes.titlesize': 7,}
plt.rcParams.update(newparams)

### Работа со звуком

Звук - это продольная волна в среде, такой как воздух или вода. Это вибрация, которая распространяется через среду в виде колебаний давления и перемещений частиц. Например, тон А - это звуковая волна частотой 440 Гц, создаваемая камертоном, динамиком, струной или другим устройством, которое заставляет воздух колебаться с заданной частотой. На компьютере звуки - это не что иное, как последовательность цифр. Данный тон может быть математически описан с помощью

$$
s(t)=A\sin(2\pi f t),
$$

где $A$ - амплитуда, $f$ - частота, а $t$ - время. В компьютере $s(t)$ оценивается в дискретные моменты времени, определяемые частотой дискретизации. Обычный аудиокод имеет частоту дискретизации 44100 Гц.

Следующая функция создает мелодию с определенной частотой, длиной, амплитудой и частотой дискретизации.

In [ ]:
def tone(frequency=440., length=1., amplitude=1., sampleRate=44100., soundType='int8'):
    """ Returns a sine function representing a tune with a given frequency.
    
    :frequency: float/int. Frequency of the tone.
    :length: float/int. Length of the tone in seconds.
    :amplitude: float/int. Amplitude of the tone.
    :sampleRate: float/int. Sampling frequency.
    :soundType: string. Type of the elements in the returned array.
    :returns: float numpy array. Sine function representing the tone.
    """
    t = np.linspace(0,length,int(length*sampleRate))
    data = amplitude*np.sin(2*np.pi*frequency*t)
    return data.astype(soundType)

Для простоты давайте создадим ностальгический 8-битный звуковой файл mono wav. Обратите внимание, что 8-разрядные образцы хранятся как uint8 в диапазоне от 0 до 255, в то время как 16-разрядные образцы хранятся как int16 в диапазоне от -32768 до 32767. Другими словами, поскольку мы создаем аудиофайл с шириной выборки 8 бит, нам нужно добавить 128 к образцам, прежде чем мы запишем их в файл. Модуль wave автоматически изменится на uint8, если это не будет сделано, так что -128 станет 128, -127 станет 129 и так далее, и звуковой файл может быть несколько искажен. Спасибо Эйстейну Хиосену за то, что он указал на это! Если мы хотим использовать 16-битные образцы, нам не нужно беспокоиться об этом. Этот ноутбук поддерживает 8, 16 и 32-разрядные образцы.

In [ ]:
# Parameters that are being used in the start of this notebook
sampleRate = 44100
sampwidth = 1          # In bytes. 1 for 8 bit, 2 for 16 bit and 4 for 32 bit
volumePercent = 50     # Volume percentage
nchannels = 1          # Mono. Only mono works for this notebook

# Some dependent variables
shift = 128 if sampwidth == 1 else 0 # The shift of the 8 bit samples, as explained in the section above.
soundType = 'i' + str(sampwidth)
amplitude = np.iinfo(soundType).min*volumePercent/100.

In [ ]:
# Parameters for a A tone lasting 1 second at a sample
frequency = 440
length = 1

# Calculate the sine function for the given parameters
data = tone(frequency, length, amplitude, sampleRate, soundType)

# Plot the function
plt.plot(data)
plt.xlim([0,2000])
plt.title('Visualization of the A tone')
plt.xlabel('Sample number')

# Open a new .wav-file in "write" mode, set file parameters
# and write to file
data += shift # Shift the samplings if we use 8 bit
with wave.open('Atone.wav', 'w') as file:
    file.setparams((nchannels, sampwidth, sampleRate, 0, 'NONE', ''))
    file.writeframes(data)

# Display the sound file
IPython.display.Audio('Atone.wav')

Давайте попробуем создать "ритм", позволив двум волнам разной частоты интерферировать. Тогда биение будет иметь частоту, равную абсолютному значению разницы в частоте двух волн, $f = \left|f_2 - f_1\right|$. Вы можете прочитать больше о частотах биений на великой [Гиперфизике](http://hyperphysics.phy-astr.gsu.edu/hbase/sound/beat.html).

In [ ]:
frequency = 400
frequency2 = 408
length = 5

# Calculate the sine function for the given parameters
data = ( tone(frequency, length, amplitude, sampleRate,soundType) +
         tone(frequency2, length, amplitude, sampleRate,soundType) )

# Plot the function
plt.plot(data)
plt.xlim([0,20000])
plt.title('Visualization of beat')
plt.xlabel('Sample number')

# Create sound file
data += shift
with wave.open('beat.wav','w') as file:
    file.setparams((nchannels, sampwidth, sampleRate, 0, 'NONE', ''))
    file.writeframes(data)

IPython.display.Audio('beat.wav')

Используя эти концепции, мы действительно можем создать простую мелодию. 

In [ ]:
# Create a "function" to translate from a given tone to its frequency
baseTone = 130.813  # The frequecy of the tone C3, bass C
tones = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'Bb', 'B',
         'Ch','C#h','Dh','D#h','Eh','Fh','F#h','Gh','G#h','Ah','Bbh','Bh']
# And use dictionary comprehension to fill in the frequencies
notes2freq = {tones[i]: baseTone*2**(i/12) for i in range(0,len(tones))}

# The meldody data saved as a list of tuples (note, duration)
l = 2.
notes = [('D',0.083*l),('D',0.083*l),('D',0.083*l),('G',0.5*l),('Dh',0.5*l),('Ch',0.083*l),
         ('B',0.083*l),('A',0.083*l),('Gh',0.5*l),('Dh',0.25*l),('Ch',0.083*l),('B',0.083*l),
         ('A',0.083*l),('Gh',0.5*l),('Dh',0.25*l),('Ch',0.083*l),('B',0.083*l),('Ch',0.083*l),
         ('A',0.5*l),('D',0.167*l),('D',0.083*l),('G',0.5*l),('Dh',0.5*l),('Ch',0.083*l),
         ('B',0.083*l),('A',0.083*l),('Gh',0.5*l),('Dh',0.25*l),('Ch',0.083*l),('B',0.083*l),
         ('A',0.083*l),('Gh',0.5*l),('Dh',0.25*l),('Ch',0.083*l),('B',0.083*l),('Ch',0.083*l),
         ('A',0.5*l),('D',0.167*l),('D',0.083*l),('E',0.375*l),('E',0.125*l),('Ch',0.125*l),
         ('B',0.125*l),('A',0.125*l),('G',0.125*l),('G',0.083*l),('A',0.083*l),('B',0.083*l),
         ('A',0.167*l),('E',0.083*l),('F#',0.25*l),('D',0.167*l),('D',0.083*l),('E',0.375*l),
         ('E',0.125*l),('Ch',0.125*l),('B',0.125*l),('A',0.125*l),('G',0.125*l),
         ('Dh',0.1875*l),('A',0.0625*l),('A',0.5*l),('D',0.167*l),('D',0.083*l),('E',0.375*l),
         ('E',0.125*l),('Ch',0.125*l),('B',0.125*l),('A',0.125*l),('G',0.125*l),('G',0.083*l),
         ('A',0.083*l),('B',0.083*l),('A',0.167*l),('E',0.083*l),('F#',0.25*l),('Dh',0.167*l),
         ('Dh',0.083*l),('Gh',0.125*l),('Fh',0.125*l),('D#h',0.125*l),('Ch',0.125*l),
         ('Bb',0.125*l),('A',0.125*l),('G',0.125*l),('Dh',0.75*l)]

# Create the file data
data = np.array([],dtype=soundType)
for note, duration in notes:
    currentFrequency = notes2freq[note]
    currentTone = tone(currentFrequency, duration, amplitude, sampleRate, soundType)
    data = np.append(data, currentTone)
data += shift
with wave.open('melody.wav','w') as file:
    file.setparams((nchannels, sampwidth, sampleRate, 0, 'NONE', ''))
    file.writeframes(data)

IPython.display.Audio('melody.wav')

Красивое.

### Теорема о выборке Найквиста-Шеннона и сглаживание

Прежде чем двигаться дальше, мы должны кратко обсудить теорему о выборке Найквиста-Шеннона. Как упоминалось ранее, когда мы имеем дело с цифровым звуком, нам необходимо сделать плавный звуковой сигнал дискретным. Это достигается путем дискретизации сигнала в дискретные моменты времени. Количество выборок в единицу времени описывается частотой дискретизации, и интуитивно понятно, что частота дискретизации должна сильно зависеть от частот сигнала.

Теорема выборки Найквиста-Шеннона гласит, что для восстановления сигнала частота дискретизации должна быть более чем в два раза больше максимальной частоты сигнала (часто называемой критерием однозначного представления) [3]. C. E. Шеннон сформулировал ее как [4]:

> Если функция $f(t)$ не содержит частот выше $W$ Гц, она полностью определяется путем указания ее координат в ряде точек, расположенных на расстоянии $1/(W/2)$ секунд друг от друга.  

Поскольку диапазон человеческого слуха составляет от 20 до 20 000 Гц, частота дискретизации 44100 Гц обычных компакт-дисков вполне понятна.

Сглаживание описывает эффект, который приводит к тому, что различные сигналы становятся неразличимыми. Например, если частота samling равна $W$, частоты $f$ и $W-f$ неразличимы. Это означает, что самая высокая частота, которую можно представить с помощью частоты дискретизации $W$, равна $W/2$, часто называемой частотой Найквиста. Более того, все частоты ниже $W/2$ будут отражаться примерно на $W/2$. Это наглядно показано ниже.

Далее мы создадим аудиофайл, в котором частота постепенно увеличивается с 0 Гц до частоты дискретизации, выбранной на уровне 5000 Гц. Как мы быстро замечаем, то, что мы слышим, - это не постепенное увеличение частоты, которого мы интуитивно ожидаем, а постепенное увеличение частоты, за которым следует постепенное снижение.

In [ ]:
sampleRate = 5000
length = .5

# Calculate the sine function for the given parameters
data = np.array([],dtype=soundType)
for frequency in np.linspace(0, sampleRate, 20):
    currentTone = tone(frequency, length, amplitude, sampleRate, soundType)
    data = np.append(data, currentTone)
data += shift
with wave.open('aliasing.wav','w') as file:
    file.setparams((nchannels, sampwidth, sampleRate, 0, 'NONE', ''))
    file.writeframes(data)

IPython.display.Audio('aliasing.wav')

Существуют способы удалить некоторые псевдонимы (сглаживание), но это не рассматривается в этой записной книжке.

### Звуки в частотной области

Если мы выполняем DFT звуковой дорожки в пространственной области [амплитуда $s(t)$], результат можно интерпретировать как функцию, описывающую $s(t)$ в соответствующей частотной области. (Это более подробно объясняется в нашей [записной книжке по DFTs](https://nbviewer.jupyter.org/url/www.numfys.net/media/notebook/mo_b2_discrete_fourier_transform.ipynb).) Таким образом, если DFT выполняется для тона A, мы получим пик при $f=440$ Гц. Кроме того, и пики будут отражаться примерно на половине частоты дискретизации (из-за теоремы о выборке Найквиста-Шеннона). Кроме того, из-за сглаживания мы получим некоторые пики при $f=(440 + 880 \cdot n)$ Гц, $n=0,1,...$. Поскольку тоны конечны и из-за ошибок численного округления мы также получим некоторое добавление шума к некоторому увеличению амплитуды, близкой к пикам. Шум и сглаживание становятся меньше, если мы используем большую ширину самлинга.

Давайте построим тон A в частотной области, взяв $\log \mathcal F [s(t)](f)$. Мы сдвигаем два квадранта и определяем начало координат с частотой, равной половине частоты дискретизации, что облегчает фильтрацию.

In [ ]:
sampleRate = 44100
length = 1
frequency = 440

plt.figure(figsize=(8,5))
subplot = 0
for sampwidth in [1,2,4]:
    # Calculate the data for the given sample width
    soundType = 'i' + str(sampwidth)
    amplitude = np.iinfo(soundType).min*volumePercent/100.
    data = tone(frequency, length, amplitude, sampleRate, soundType)
    # Use DFT to tranform the data into the frequency domain
    dataFreq = np.log(fft.fftshift(fft.fft(data)))
    # Plot the results
    subplot += 1
    plt.subplot(3,1,subplot)
    plt.plot(np.linspace(-0.5*sampleRate, 0.5*sampleRate, len(dataFreq)), dataFreq)
    plt.xlim([-0.5*sampleRate, 0.5*sampleRate])
    plt.title('Frequency domain of the %d bit A tone' %(sampwidth*8))
plt.xlabel('Frequency, [Hz]')
plt.tight_layout()


Из концепции, представленной до сих пор, легко понять, как можно отфильтровать определенные частоты.

### Пример: Фильтрация определенных частот

Теперь мы собираемся отфильтровать 16-битный звуковой файл [`spock_bad.wav`](https://www.numfys.net/media/spock_bad.wav).

In [ ]:
filename = 'spock_bad'
IPython.display.Audio(filename + '.wav')

In [ ]:
with wave.open(filename + '.wav', 'rb') as file:
    data = file.readframes(-1)
    sampleRate = file.getframerate()
    sampwidth = file.getsampwidth()
    
soundType = 'i' + str(sampwidth)

data = np.fromstring(data, soundType)
n = len(data)

# Perform the DFT
dataFreq = fft.fftshift(fft.fft(data))

# Plot the sound file in the spatial domain
plt.subplot(2,1,1)
plt.plot(np.linspace(0, n/sampleRate, n), data)
plt.title('Spatial domain of %s' % filename)
plt.xlabel('Time, [s]')
plt.xlim([0, n/sampleRate])

# Plot the sound file in the frequency domain
plt.subplot(2,1,2)
plt.plot(np.linspace(-0.5*sampleRate, 0.5*sampleRate, n), np.log(np.abs(dataFreq)))
plt.title('Frequency domain of %s' % filename)
plt.xlabel('Frequency, [Hz]')
plt.xlim([-0.5*sampleRate, 0.5*sampleRate])
plt.tight_layout()

Мы наблюдаем пики на частотах около $f=\{100, 200, 1000, 5000\}$ Хз. Поэтому мы пытаемся отфильтровать их и прислушаться к результату.

In [ ]:
w = 100  # Width of the removing intervals
# Filter out the frequencies in the frequency domain
for f in [100, 200, 1000, 5000]:
    dataFreq[int(n/2 + n*f/sampleRate - w) : int(n/2 + n*f/sampleRate + w)] = 0
    dataFreq[int(n/2 - n*f/sampleRate - w) : int(n/2 - n*f/sampleRate + w)] = 0

# Transform the filtered frequency domain back to the spatial domain
data = fft.ifft(fft.fftshift(dataFreq))
data = data.astype(soundType)

# Save the filtered data to a new file
with wave.open(filename + '_filtered.wav', 'w') as A, \
    wave.open(filename + '.wav', 'rb') as file:
    A.setparams(file.getparams())
    A.writeframes(data.real)

# Plot the filtered sound in the spatial domain
plt.subplot(2,1,1)
plt.plot(np.linspace(0, n/sampleRate, n), data)
plt.title('Spatial domain of %s' % (filename + '_filtered'))
plt.xlabel('Time, [s]')
plt.xlim([0, n/sampleRate])

# Plot the filtered sound in the frequency domain
plt.subplot(2,1,2)
plt.plot(np.linspace(-0.5*sampleRate, 0.5*sampleRate, n), np.log(np.abs(dataFreq)))
plt.title('Frequency domain of %s' % (filename + '_filtered'))
plt.xlabel('Frequency, [Hz]')
plt.xlim([-0.5*sampleRate, 0.5*sampleRate])
plt.tight_layout()

# Play the filtered sound file
IPython.display.Audio(filename + '_filtered.wav')

### Пример: Фильтрация низких и высоких частот

В этом примере мы проверим, что делает идеальный фильтр нижних частот и идеальный фильтр высоких частот со звуковым файлом. Эти фильтры удаляют сигналы с частотами выше частоты среза и ниже частоты среза соответственно. Эти фильтры чаще всего неидеальны, но эти типы фильтров здесь не обсуждаются. Фильтры ослабляют частоты выше или ниже заданного значения среза. Мы используем звуковой файл [`vena.wav`](https://www.numfys.net/media/vena.wav).

In [ ]:
filename = 'vena.wav'
IPython.display.Audio(filename)

In [ ]:
def idealLowpass(data, cutoff, sampleRate=44100):
    """ Removes all frequencies above a given cutoff value of
    a mono sound file described by an array of data and a samle
    frequency.
    
    :data: float numpy array. The data of a sound file.
    :cutoff: float/int. Cutoff value.
    :sampleRate: int. The sampling frequency of the sound file.
    :returns: float numpy array. Filtered sound data.
    """
    n = len(data)
    dataFreq = fft.fftshift(fft.fft(data))
    dataFreq[0 : int(n/2 -n*cutoff/sampleRate)] = 0
    dataFreq[int(n/2 + n*cutoff/sampleRate) : len(dataFreq)] = 0
    data = fft.ifft(fft.fftshift(dataFreq))
    return data.real

def idealHighpass(data, cutoff, sampleRate):
    """ Removes all frequencies below a given cutoff value of
    a mono sound file described by an array of data and a samle
    frequency.
    
    :data: float numpy array. The data of a sound file.
    :cutoff: float/int. Cutoff value.
    :sampleRate: int. The sampling frequency of the sound file.
    :returns: float numpy array. Filtered sound data.
    """
    n = len(data)
    dataFreq = fft.fftshift(fft.fft(data))
    dataFreq[int(n/2 - n*cutoff/sampleRate) : int(n/2 + n*cutoff/sampleRate)] = 0
    data = fft.ifft(fft.fftshift(dataFreq))
    return data.real

In [ ]:
# Read data from sound file
with wave.open(filename, 'rb') as file:
    data = file.readframes(-1)
    sampleRate = file.getframerate()
    sampwidth = file.getsampwidth()
soundType = 'i' + str(sampwidth)
data = np.fromstring(data, soundType)
n = len(data)

Мы начинаем с применения фильтра нижних частот.

In [ ]:
# Perform the lowpass filter
cutoff = 500
dataLowpass = idealLowpass(data, cutoff, sampleRate)

# Save the filtered sound
with wave.open('Vena_lowpass.wav', 'w') as A, \
    wave.open(filename, 'rb') as file:
    A.setparams(file.getparams())
    A.writeframes(dataLowpass.astype(soundType))

# Play the filtered sound
IPython.display.Audio('Vena_lowpass.wav')

Как вы можете слышать, все высокие частоты исчезли, и мы остались с басовыми тонами. Обратите внимание, например, что мы больше не слышим высоких шляп и бубна.

Теперь мы применяем фильтр высоких частот.

In [ ]:
# Perform the highpass filter
cutoff = 500
dataHighpass = idealHighpass(data, cutoff, sampleRate)

# Save the filtered sound
with wave.open('Vena_highpass.wav', 'w') as A, \
    wave.open(filename, 'rb') as file:
    A.setparams(file.getparams())
    A.writeframes(dataHighpass.astype(soundType))

# Play the filtered sound
IPython.display.Audio('Vena_highpass.wav')

Теперь все наоборот; все низкие частоты исчезли, и мы больше не слышим басовых тонов.

### Пример: Удаление белого шума (случайный шум)

В этом примере мы собираемся отфильтровать белый шум, используя тригонометрическое приближение наименьших квадратов. Это приближение объясняется в нашей записной книжке [trigonometric interpolation](https://nbviewer.jupyter.org/url/www.numfys.net/media/notebooks/mo_curve2_trigonometric_interpolation.ipynb).

In [ ]:
def leastSquaresTrig(x, m, N):
    """ Calculates a trigonometric polynomial of degree n/2 (n even) or
    (n-1)/2 (n odd) that interpolates a set of n real data points. The
    data points can be written as (t_i,x_i), i=0,1,2,..., where t_i are 
    equally spaced in the interval [c,d].
    
    :x: complex or float numpy array. Data points.
    :c: float. Interval start. t[0].
    :d: float. Interval end. t[n-1].
    :returns: float numpy array.
    """
    n = len(x)
    if not 0<=m<=n<=N:
        raise ValueError('Is 0 <= m <= n <= N?')
    y = fft.fft(x)
    yp = np.zeros(N, np.complex64)
    m2 = int(m/2)
    yp[0:m2] = y[0:m2]
    yp[N - m2 + 1:N] = y[n - m2 + 1:n]
    if (m % 2):
        yp[m2] = y[m2]
    else:
        yp[m2] = np.real(y[m2])
    if m<n and m>0:
        yp[N-m2] = yp[m2]
    return np.real(fft.ifft(yp))*N/n

We are using the sound file [`comet_noise.wav`](https://www.numfys.net/media/comet_noise.wav).

In [ ]:
filename = 'comet_noise.wav'
IPython.display.Audio(filename)

In [ ]:
with wave.open(filename, 'rb') as file:
    data = file.readframes(-1)
    sampleRate = file.getframerate()
    sampwidth = file.getsampwidth()
soundType = 'i' + str(sampwidth)
data = np.fromstring(data, soundType)
data = leastSquaresTrig(data, len(data)/14, len(data))

with wave.open('comet_filtered.wav', 'w') as A, \
    wave.open(filename, 'rb') as file:
    A.setparams(file.getparams())
    A.writeframes(data.astype(soundType))

IPython.display.Audio('comet_filtered.wav')

Обратите внимание, что белый шум больше не слышен, но это за счет качества.

### References

All the sound files are collected from http://tos.trekcore.com/.

[1] H. P. Langtangen: "A primer on scientific programming with Python", 4th edition, p. 621-627, Springer 2014  
[2] R Nave: "Beats", http://hyperphysics.phy-astr.gsu.edu/hbase/sound/beat.html  
[3] C. E. Shannon: "Communication in the presence of noise", Proc. Institute of Radio Engineers, vol. 37, no.1, p. 10–21, 1949.  
[4] Wikipedia: "Audio bit depth", https://en.wikipedia.org/wiki/Audio_bit_depth, 10th May 2016 [acquired: 11th May 2016]  
[5] Wikipedia: "Nyquist–Shannon sampling theorem", https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem, 9th May 2016 [acquired: 11th May 2016]